# Candlestick body & pattern & return relationship

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import os
from pathlib import Path
import pandas_ta as ta
import pspriceaction.price_action as pa
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [2]:
data = []
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
algo_dir = current_dir.parent.parent
transform_csv_file = str(current_dir) + '/VN30F1M_5minutes_pattern.csv'
if os.path.isfile(transform_csv_file):
    data = pd.read_csv(transform_csv_file, index_col='Date', parse_dates=True)
    data['model'] = data['model'].fillna(value='')
else:
    csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
    is_file = os.path.isfile(csv_file)
    if is_file:
        dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
    else:
        dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)
    data = dataset.copy()
    data = pa.pattern_modeling(data)
    data.to_csv(transform_csv_file)

### Calculating return

In [3]:
def has_bullish_pattern(model):
    if "bullish" in model or "rising" in model:
        return True
    return False

def has_bearish_pattern(model):
    if "bearish" in model or "falling" in model:
        return True
    return False

In [4]:
%%time
data['return'] = ''
data['signal'] = ''
data['exit_time'] = ''
# Stoploss at x0 pips
sl_step = 3
# Takeprofit at y0 pips(R/R = 1/3)
tp_step = 9
for i, row in data.iterrows():
    if has_bullish_pattern(row['model']):
        # Long signal
        data.at[i, 'signal'] = 'long'
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        entry_price = row['Close']
        data_to_end_day = data[(data.index > current_time) & (data.index < current_date+' 14:30:00')]
        max_price = 0
        exit_time = ''
        for k, wrow in data_to_end_day.iterrows():
            if wrow['Low'] < entry_price and wrow['Low'] < entry_price - sl_step:
                # Stop loss
                momentum = -sl_step
                exit_time = wrow.name
                break
            else:
                if wrow['High'] > entry_price + tp_step:
                    # Take profit
                    momentum = tp_step
                    exit_time = wrow.name
                    break
                else:
                    # Close at 02:25PM
                    momentum = wrow['Close'] - entry_price
                    exit_time = wrow.name
        data.at[i, 'return'] = momentum
        data.at[i, 'exit_time'] = exit_time
    elif has_bearish_pattern(row['model']):
        # Short signal
        data.at[i, 'signal'] = 'short'
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        entry_price = row['Close']
        data_to_end_day = data[(data.index > current_time) & (data.index < current_date+' 14:30:00')]
        min_price = 10000
        exit_time = ''
        for k, wrow in data_to_end_day.iterrows():
            if wrow['High'] > entry_price and wrow['High'] > entry_price + sl_step:
                # Stop loss
                momentum = -sl_step
                exit_time = wrow.name
                break
            else:
                if wrow['Low'] < entry_price - tp_step:
                    # Take profit
                    momentum = tp_step
                    exit_time = wrow.name
                    break
                else:
                    # Close at 02:25PM
                    momentum = entry_price - wrow['Close']
                    exit_time = wrow.name
        data.at[i, 'return'] = momentum
        data.at[i, 'exit_time'] = exit_time

CPU times: user 19.6 s, sys: 2.88 ms, total: 19.6 s
Wall time: 19.6 s


### Return Analytics

In [5]:
has_return = data[data['return'] != '']
#----
long_return = has_return[has_return['signal'] == 'long']
short_return = has_return[has_return['signal'] == 'short']
#----
negative_return = has_return[has_return['return'] < 0]
positive_return = has_return[has_return['return'] > 0]
#----
short_negative_return = short_return[short_return['return'] < 0]
short_positive_return = short_return[short_return['return'] > 0]
long_negative_return = long_return[long_return['return'] < 0]
long_positive_return = long_return[long_return['return'] > 0]

## Pattern model & return relationship

### Group by model

In [6]:
has_return_group = has_return[['return']].groupby([has_return.model])

### Sum return

In [7]:
sum_group = has_return_group.sum()
sum_group['return'].sort_values()

model
bullish_separating_line                 -53.5
bullish_gap, rising_n                     3.1
bearish_gap, falling_three                4.9
bearish_gap, falling_n                      6
bullish_gap, rising_three                 8.8
rising_n                                 63.6
falling_three                            83.2
bullish_gap, fair_value_rising_gap       83.8
bearish_gap                             104.6
bearish_gap, fair_value_falling_gap     124.8
falling_n                               125.8
bullish_gap                             127.2
rising_three                            158.1
bullish_neck                            260.3
bearish_neck                            580.8
fair_value_falling_gap                 2188.0
fair_value_rising_gap                  2639.7
Name: return, dtype: object

### Mean group

In [8]:
mean_group = has_return_group.mean()
mean_group['return'].sort_values()

model
bullish_separating_line               -0.121315
bullish_neck                           0.170689
falling_three                           0.34958
bearish_neck                           0.384127
bullish_gap, fair_value_rising_gap     0.462983
bearish_gap                            0.484259
rising_n                               0.578182
bullish_gap                            0.583486
fair_value_falling_gap                 0.647721
fair_value_rising_gap                  0.724396
rising_three                           0.738785
bearish_gap, fair_value_falling_gap    0.866667
bearish_gap, falling_three                 0.98
falling_n                              1.093913
bullish_gap, rising_n                      1.55
bullish_gap, rising_three              2.933333
bearish_gap, falling_n                      3.0
Name: return, dtype: object

### Count return

In [9]:
has_positive_return_group = positive_return[['return']].groupby([positive_return.model])

In [10]:
count_group_positive = has_positive_return_group.count()
count_group_positive['return'].sort_values()

model
bearish_gap, falling_n                    1
bullish_gap, rising_n                     1
bearish_gap, falling_three                2
bullish_gap, rising_three                 3
rising_n                                 41
falling_n                                52
bearish_gap, fair_value_falling_gap      64
bullish_gap, fair_value_rising_gap       73
bullish_gap                              81
bearish_gap                              83
rising_three                             90
falling_three                            91
bullish_separating_line                 149
bullish_neck                            548
bearish_neck                            555
fair_value_falling_gap                 1333
fair_value_rising_gap                  1526
Name: return, dtype: int64

In [11]:
has_negative_return_group = negative_return[['return']].groupby([negative_return.model])

In [12]:
count_group_negative = has_negative_return_group.count()
count_group_negative['return'].sort_values()

model
bearish_gap, falling_n                    1
bullish_gap, rising_n                     1
bearish_gap, falling_three                3
falling_n                                63
rising_n                                 69
bearish_gap, fair_value_falling_gap      80
bullish_gap, fair_value_rising_gap      108
rising_three                            123
bearish_gap                             131
bullish_gap                             137
falling_three                           146
bullish_separating_line                 290
bearish_neck                            946
bullish_neck                            972
fair_value_falling_gap                 2028
fair_value_rising_gap                  2102
Name: return, dtype: int64